In [1]:
ls

HW_10.10_DATASET_ANALYSE.ipynb  data/


In [2]:
ls ./data/

riiid-test-answer-prediction/     riiid-test-answer-prediction.zip


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

## Проверяем какие данные хранятся в таблице

In [19]:
reader = pd.read_csv('data/train.csv', iterator=True, chunksize=512)
reader.get_chunk(100)

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False
...,...,...,...,...,...,...,...,...,...,...
95,95,835457,2746,484,0,19,0,1,20000.0,True
96,96,0,5382,5000,0,0,0,1,NaN,NaN
97,97,39828,5382,3944,0,1,1,0,24000.0,False
98,98,132189,5382,217,0,2,0,1,35000.0,False


In [20]:
train_dt = pd.read_csv('data/train.csv', usecols=['timestamp', 'user_id', 'content_id', 'content_type_id', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation'])

In [22]:
train_dt.shape[0]

101230332

In [24]:
question_dt = pd.read_csv('data/questions.csv')

In [25]:
question_dt.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [26]:
question_dt.shape[0]

13523

In [27]:
lectures_dt = pd.read_csv('data/lectures.csv')

In [28]:
lectures_dt.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [29]:
train_dt_copy = train_dt.copy()

## Подготовка данных

In [31]:
train_dt_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 7 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int64  
 2   content_id                      int64  
 3   content_type_id                 int64  
 4   answered_correctly              int64  
 5   prior_question_elapsed_time     float64
 6   prior_question_had_explanation  object 
dtypes: float64(1), int64(5), object(1)
memory usage: 5.3+ GB


#### Преобразуем колонку timestamp в более наглядное представление

In [42]:
train_dt_copy.head()

,timestamp,user_id,content_id,content_type_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,0,1,NaN,NaN
1,56943,115,5716,0,1,37000.0,False
2,118363,115,128,0,1,55000.0,False
3,131167,115,7860,0,1,19000.0,False
4,137965,115,7922,0,1,11000.0,False


In [44]:
train_dt_copy['seconds solution time'] = round(train_dt_copy['timestamp'] / 1000)

In [46]:
train_dt_copy = train_dt_copy.drop('timestamp', axis = 1)

### Для удобства замаппим тип контента на названия: Lecture / Question

In [52]:
train_dt_copy['content type'] = train_dt_copy['content_type_id'].apply(lambda x: 'lecture' if x==1 else 'question')

In [58]:
train_dt_copy[train_dt_copy['content type'] == 'question'].head(1000)

,user_id,content_id,content_type_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,seconds solution time,content type
0,115,5692,0,1,NaN,NaN,0.0,question
1,115,5716,0,1,37000.0,False,57.0,question
2,115,128,0,1,55000.0,False,118.0,question
3,115,7860,0,1,19000.0,False,131.0,question
4,115,7922,0,1,11000.0,False,138.0,question
...,...,...,...,...,...,...,...,...
1014,13134,8740,0,1,4000.0,True,15094542.0,question
1015,13134,5223,0,0,6000.0,True,15094575.0,question
1016,13134,6091,0,1,14000.0,True,15094590.0,question
1017,13134,9237,0,1,6000.0,True,15094616.0,question


### Удалим ненужные строки

In [70]:
train_dt_copy = train_dt_copy.drop('content_type_id', axis = 1)

In [72]:
train_dt_copy.head()

,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,seconds solution time,content type
0,115,5692,1,NaN,NaN,0.0,question
1,115,5716,1,37000.0,False,57.0,question
2,115,128,1,55000.0,False,118.0,question
3,115,7860,1,19000.0,False,131.0,question
4,115,7922,1,11000.0,False,138.0,question


### Для контента типа question добавим раздел теста

In [77]:
traint_dt_copy_copy = train_dt_copy.copy()

In [82]:
lectures_dt[lectures_dt['lecture_id'] == 5692]

,lecture_id,tag,part,type_of


In [88]:
train_dt_copy['part'] = train_dt_copy['content_id'].map(lectures_dt['part'])

In [89]:
train_dt_copy.head()

,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,seconds solution time,content type,part
0,115,5692,1,NaN,NaN,0.0,question,NaN
1,115,5716,1,37000.0,False,57.0,question,NaN
2,115,128,1,55000.0,False,118.0,question,5.0
3,115,7860,1,19000.0,False,131.0,question,NaN
4,115,7922,1,11000.0,False,138.0,question,NaN


In [98]:
def map_part_id_to_type(part_id):
    if part_id is None:
        return
    elif part_id >= 1 and part_id <= 4:
        return 'Listening'
    elif part_id > 4 and part_id < 8:
        return 'Reading'
    else:
        return

In [101]:
train_dt_copy['part'] = train_dt_copy['part'].apply(map_part_id_to_type)

In [105]:
train_dt_copy.describe(include = {'object'})

,prior_question_had_explanation,content type,part
count,100837826,101230332,6427635
unique,2,2,2
top,True,question,Reading
freq,89685560,99271300,4155198


In [109]:
train_dt_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 8 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   user_id                         int64  
 1   content_id                      int64  
 2   answered_correctly              int64  
 3   prior_question_elapsed_time     float64
 4   prior_question_had_explanation  object 
 5   seconds solution time           int64  
 6   content type                    object 
 7   part                            object 
dtypes: float64(1), int64(4), object(3)
memory usage: 6.0+ GB


In [108]:
train_dt_copy['seconds solution time'] = train_dt_copy['seconds solution time'].astype(int)

In [111]:
train_dt_copy = train_dt_copy.fillna(0)

In [132]:
train_dt_copy['prior_question_elapsed_time'] = train_dt_copy['prior_question_elapsed_time'].astype(int)

In [117]:
train_dt_copy['prior_question_elapsed_time']  = round(train_dt_copy['prior_question_elapsed_time'] / 1000)

In [133]:
train_dt_copy.head()

,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,seconds solution time,content type,part
0,115,5692,1,0,0,0,question,0
1,115,5716,1,37,False,57,question,0
2,115,128,1,55,False,118,question,Reading
3,115,7860,1,19,False,131,question,0
4,115,7922,1,11,False,138,question,0


In [123]:
train_dt_copy[train_dt_copy['part'] != 0].describe(include = {'object'})

,prior_question_had_explanation,content type,part
count,6427635,6427635,6427635
unique,2,2,2
top,True,question,Reading
freq,5695921,6391734,4155198


#### Средняя скорость ответа

In [127]:
round(train_dt_copy['prior_question_elapsed_time'].mean())

25

#### Наибольшая средняя скорость ответа на вопрос

In [141]:
#round(train_dt_copy[train_dt_copy['prior_question_elapsed_time'] > 0].max())
train_dt_copy['prior_question_elapsed_time'].median()

21.0

In [149]:
train_dt_copy['prior_question_elapsed_time'].quantile(0.5)

21.0

In [151]:
train_dt_copy[train_dt_copy['prior_question_had_explanation'] == True]['prior_question_elapsed_time'].quantile(0.99)

98.0

In [152]:
train_dt_copy[train_dt_copy['part'] == 'Reading']['prior_question_elapsed_time'].quantile(0.99)

58.0

In [153]:
train_dt_copy[train_dt_copy['part'] == 'Listening']['prior_question_elapsed_time'].quantile(0.99)

59.0

In [155]:
train_dt_copy[(train_dt_copy['part'] == 'Listening') & (train_dt_copy['prior_question_had_explanation'] == True)]['prior_question_elapsed_time'].quantile(0.99)

55.0

In [156]:
train_dt_copy[(train_dt_copy['part'] == 'Reading') & (train_dt_copy['prior_question_had_explanation'] == True)]['prior_question_elapsed_time'].quantile(0.99)

55.0

In [157]:
train_dt_copy.head()

,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,seconds solution time,content type,part
0,115,5692,1,0,0,0,question,0
1,115,5716,1,37,False,57,question,0
2,115,128,1,55,False,118,question,Reading
3,115,7860,1,19,False,131,question,0
4,115,7922,1,11,False,138,question,0


In [160]:
train_dt_copy[train_dt_copy['answered_correctly'] != -1]['answered_correctly'].value_counts()

1    65244627
0    34026673
Name: answered_correctly, dtype: int64

In [166]:
train_dt_copy[train_dt_copy['answered_correctly'] == 1]['answered_correctly'].shape[0] / train_dt_copy[train_dt_copy['answered_correctly'] != -1]['answered_correctly'].shape[0]

0.6572355454194717

In [167]:
train_dt_copy[(train_dt_copy['answered_correctly'] == 1) & (train_dt_copy['prior_question_had_explanation'] == True)]['answered_correctly'].shape[0] / train_dt_copy[(train_dt_copy['answered_correctly'] != -1) & (train_dt_copy['prior_question_had_explanation'] == True)]['answered_correctly'].shape[0]

0.6731885043701572

In [168]:
train_dt_copy[(train_dt_copy['answered_correctly'] == 1) & (train_dt_copy['prior_question_had_explanation'] == False)]['answered_correctly'].shape[0] / train_dt_copy[(train_dt_copy['answered_correctly'] != -1) & (train_dt_copy['prior_question_had_explanation'] == False)]['answered_correctly'].shape[0]

0.5079773705525082

### <span style='color:Blue'> Итог анализа: </span>

1. Наибольший контект, представленный в процессе обучения - вопросы теста <br>
2. На большинство вопросов получено разъяснение
3. Наибольшее количество вопросов в тесте содержится в разделе: Чтение
5. Наибольшее среднее время ответа на каждый вопрос - 5 минут
6. Стандартное отклонение 21 секунда
7. В 99% случаев среднее время ответа на вопрос меньше 96 секунд
8. В 99% случаев среднее время ответа при разобранном предыдущем вопрос больше на 2 секунды
9. В 99% случаев в тесте на чтение время ответа равно 58 секунд
10. В 99% случаев в тесте на аудирование время ответа равно 59 секунд
11. В 99% случаев время ответа на вопрос было на 4 секунды быстрее в аудировании и чтении, если был разобран список вопросов в предыдущем тесте
12. Доля правильных ответов 65 процентов
13. Доля правильных ответов после разъяснения предыдущего теста 67%
14. Доля правильных ответов без разъяснения предыдущего теста 51%
